# Introduction to Julia and JuliaCall

# R is slow, how about writing functions in C/Cpp?

## Pros

* Faster functions.

## Cons

* Need to learn a language that is (very) different from R, and is quite low level (C) or quite complicated (cpp).

* It doesn't compose well.

## An example from R itself

R itself already uses lots of functions in C, but is still slow....

R: `sin(sqrt(a))` ## a is some vector

1. R: evaluate `sqrt(a)` -> call C function
2. C: create a new vector -> do the calculation -> give the result `b = sqrt(a)` back to R
3. R: evaluate `sin(b)` -> call C function
4. C: create a new vector -> do the calculation -> give the result `c = sin(b)` back to R.

This is a *sub*optimal.

Why not
1. R: evaluate `sqrt(a)` -> call C function
2. C: create a new vector -> do the calculation -> give the result `b = sin(sqrt(a))` back to R?

If we want to achive this, we need to write a C function called `sin_sqrt` and then use it from R. And maybe we also need to write `sqrt_sin`, `exp_sin`, `sin_exp`, `cos_sin`, `sin_cos` ....



# Just In Time (JIT) Compiling

* Functions need to be executed will be compiled first.

* JIT is widely used, in Julia, in your browser (Google V8), in MATLAB, in Python (numba), and in R.

## MATLAB and Python's JIT

### Pros

* Fast when they work.

* Being optimized for much longer than Julia.

### Cons

* It's limited.

* It doesn't compose well.

# Julia's advantage

Instead of the case in R/Python/MATLAB, where you have the language first, and then consider the optimization (JIT), Julia's design is to make JIT easier.

* Not limited: users can define their own types, and Julia will try to do JIT for users.

* Compose well: if Julia find further optimization opportunities, it will do it automatically.

## An example

Julia: `sin.(sqrt.(a))` ## a is some vector

1. create a new vector `b`
2. for each element `x` in `a` do the calculation of `sin(sqrt(x))` 
3. stores the result in `b`


# Julia

## Official website

<https://julialang.org/>

## IDEs

* Juno in [Atom](https://atom.io/)

* [Visual Studio Code](https://code.visualstudio.com/)

* [Jupyter](https://jupyter.org/try)

* [Julia Box](https://juliabox.com/)

## Version choice


* Convex optimization: [Convex.jl](https://github.com/JuliaOpt/Convex.jl)

* Optimization: [Optim.jl](https://github.com/JuliaNLSolvers/Optim.jl/), [JuMP.jl](https://github.com/JuliaOpt/JuMP.jl)

* Differential equations: [DifferentialEquations.jl](https://github.com/JuliaDiffEq/DifferentialEquations.jl)

* Automatic differentiation: [ForwardDiff.jl](https://github.com/JuliaDiff/ForwardDiff.jl), [ReverseDiff.jl](https://github.com/JuliaDiff/ReverseDiff.jl)

* Mixed Effects Models: [MixedModels.jl](https://github.com/dmbates/MixedModels.jl)

* Artificial Intelligence: [Flux.jl](https://github.com/FluxML/Flux.jl), [TensorFlow.jl](https://github.com/malmaud/TensorFlow.jl)


In [1]:
## using Pkg; Pkg.add("JuMP"); Pkg.add("Ipopt")
using JuMP
using Ipopt

p = 3
m = Model(solver = IpoptSolver())
@variable(m, x[1:p])
@objective(m, Min, sum(x.*x))
status = solve(m)
getvalue(x[1:p])


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

3-element Array{Float64,1}:
 0.0
 0.0
 0.0

In [2]:
## using Pkg; Pkg.add("ForwardDiff")
using ForwardDiff

function NewtMin(f, x0, eps)
    fgrad = x-> ForwardDiff.gradient(f, x)
    fhess = x-> ForwardDiff.hessian(f, x)
    oldval = f(x0)
    newx = x0 - fhess(x0)\fgrad(x0)
    newval = f(newx)
    while abs(newval - oldval) > eps
        oldval = newval
        newx = newx - fhess(newx)\fgrad(newx)
        newval = f(newx)
    end
    return newx
end

f(x) = sum(x.^2)

NewtMin(f, [1., 1., 1.], 1e-6)

3-element Array{Float64,1}:
 0.0
 0.0
 0.0

In [3]:
## using Pkg; Pkg.add("Optim")
using Optim

r = Optim.optimize(f, [1., 1., 1.])
Optim.minimizer(r)

3-element Array{Float64,1}:
 -5.0823878310249317e-5
  9.262030970117879e-6 
 -1.2828696727829803e-5

# Advanced: multiple dispatch

* Similar to S3 system in R: `print.lm`, `anova.lm`, etc, but works on multiple arguments, and is **fast**!

* A simple example:

    julia> function f(x::Integer) x+1 end
    f (generic function with 1 method)

    julia> function f(x::AbstractFloat) 3 * x end
    f (generic function with 2 methods)


# More advanced

* `@code_llvm`

* `@code_warn_type`

* `@trace` from [Traceur.jl](https://github.com/MikeInnes/Traceur.jl)

## Some more advanced and comprehensive introductions

* <https://github.com/johnfgibson/whyjulia>

* <http://ucidatascienceinitiative.github.io/IntroToJulia/Html/WhyJulia>

# JuliaCall

## Website

* <https://github.com/Non-Contradiction/JuliaCall>

* <https://cran.r-project.org/web/packages/JuliaCall/index.html>